# Life Time Value Analysis

Lifetime Value es el término que se utiliza para determinar el valor que un cliente aporta a un negocio durante toda la vida útil de la empresa.

<img src="https://www.antevenio.com/wp-content/uploads/2018/12/lifetime-value.png" width="600" height="400" />

___
> ### *Procedimiento*
    - Identificar buenos clientes con LTV: Se necesita el valor de cada orden, numero de ordenes, tiempo que pasó ente la orden 3 y la orden N, Utilidad.
    - Valores Fijos: Tasa de retención de 70%, Tasa de descuento fija de 12% anual.
    
> ### *Obtención de información ajena al LTV*
    Si bien el LTV se puede calcular cuándo el cliente ya está establecido. Lo podemos calcular con la información hasta su primera orden?
    - Numero de eventos registrados
    - Sesiones
    - Datos de registro
    - Encuestras
    - Respuestas a push
    - Tarjetas
    - Versión de celular
    - ETC

> ### *Regresión Lineal*
    Con las variables ajenas al LTV predecir el LTV. Revisar que es significativo y que no.

In [54]:
# Librerías
import pandas as pd
import json     
import snowflake.connector
import datetime

#----------CONNECTION TO SNOWFLAKE-------------
SF_ACCOUNT = 'gfa04036.us-east-1'
SF_WH = 'TRANSFORMING'
SF_USERNAME = 'DBT_USER'
SF_PASSWORD = '2C>`8Q!8y*Sz]h/):Xxy&WNJv'

# Connecting to Snowflake using the default authenticator
ctx = snowflake.connector.connect(
  user=SF_USERNAME,    #username,
  password=SF_PASSWORD,    #password,
  account=SF_ACCOUNT,
  warehouse=SF_WH,
  database='ANALYTICS',
  schema='PROD_STAGING'
)

cur=ctx.cursor()

In [55]:
# Reading data
sql =   ''' SELECT * FROM (
SELECT
  "prod_modeled_orders"."SOURCE_COUNTRY",
  "prod_modeled_orders"."ID",
  "prod_modeled_orders"."CLIENT_ID",
  "prod_modeled_orders"."CREATED_AT",
  "prod_modeled_products"."PRICE_USD",
  "prod_modeled_products"."GOAL_CATEGORY_NAME",
  "prod_modeled_package_details"."DELIVERED_QUANTITY"
FROM
  "PROD_MODELED"."ORDERS" "prod_modeled_orders"
  LEFT JOIN "PROD_MODELED"."ORDER_DETAILS" "prod_modeled_order_details" ON CONCAT("prod_modeled_order_details"."ORDER_ID", '-', "prod_modeled_order_details"."SOURCE_COUNTRY") = CONCAT("prod_modeled_orders"."ID", '-', "prod_modeled_orders"."SOURCE_COUNTRY")
  LEFT JOIN "PROD_MODELED"."SHIPMENT_DETAILS" "prod_modeled_shipment_details" ON CONCAT("prod_modeled_shipment_details"."ORDER_ID", '-', "prod_modeled_shipment_details"."PRODUCT_ID", '-', "prod_modeled_shipment_details"."SOURCE_COUNTRY") = CONCAT("prod_modeled_order_details"."ORDER_ID", '-', "prod_modeled_order_details"."PRODUCT_ID", '-', "prod_modeled_order_details"."SOURCE_COUNTRY")
  LEFT JOIN "PROD_MODELED"."PACKAGE_DETAILS" "prod_modeled_package_details" ON CONCAT("prod_modeled_package_details"."SHIPMENT_ID", '-', "prod_modeled_package_details"."PRODUCT_ID", '-', "prod_modeled_package_details"."SOURCE_COUNTRY") = CONCAT("prod_modeled_shipment_details"."SHIPMENT_ID", '-', "prod_modeled_shipment_details"."PRODUCT_ID", '-', "prod_modeled_shipment_details"."SOURCE_COUNTRY")
  LEFT JOIN "PROD_MODELED"."PRODUCTS" "prod_modeled_products" ON CONCAT("prod_modeled_order_details"."PRODUCT_ID",'-', "prod_modeled_order_details"."WAREHOUSE_ID", '-', "prod_modeled_order_details"."SOURCE_COUNTRY") = CONCAT("prod_modeled_products"."ID",'-',"prod_modeled_products"."WAREHOUSE_ID", '-', "prod_modeled_products"."SOURCE_COUNTRY")
  LEFT JOIN "PROD_MODELED"."PACKAGES" "prod_modeled_packages" ON CONCAT("prod_modeled_package_details"."PACKAGE_ID", '-', "prod_modeled_package_details"."SOURCE_COUNTRY") = CONCAT("prod_modeled_packages"."PACKAGE_ID", '-',"prod_modeled_packages"."SOURCE_COUNTRY")
WHERE
  ("prod_modeled_orders"."STATUS" = CAST ( 'completed' AS TEXT )) AND
  ("prod_modeled_packages"."REJECTED_AT" IS NULL) AND
  ("prod_modeled_package_details"."DELIVERED_QUANTITY" > 0) AND
  ("prod_modeled_orders"."SEQUENTIAL" NOT IN (1,2) AND
  ("prod_modeled_products"."PRICE_USD" IS NOT NULL))
ORDER BY
  "prod_modeled_orders"."CREATED_AT"
) '''

cur.execute(sql)

# Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
orders = cur.fetch_pandas_all()
orders.reset_index(inplace=True)
orders = orders.to_dict(orient = 'index')

print(len(orders))

367475


In [56]:
# Parámetros para margen de cada cliente.
margins = {
    "Banos y cocinas": 0.14,
    "Cementos": 0.045,
    "Combos": 0.04,
    "Derivados del acero": 0.11,
    "Electricos": 0.03,
    "Herramientas": 0.12, 
    "Ladrillos, ferreteria, aseo y jardin": 0.13, 
    "Pinturas": 0.06, 
    "Plomeria": 0.05, 
    "Quimicos": 0.085,
    "Varilla": 0.015
}

consolidatedData = {}
for i in orders:
    client_id = orders[i]['CLIENT_ID']
    source_country = orders[i]['SOURCE_COUNTRY']
    if (client_id,source_country) in consolidatedData:
        consolidatedData[client_id,source_country]['GMV'] += orders[i]['PRICE_USD']*orders[i]['DELIVERED_QUANTITY']
        try:
            consolidatedData[client_id,source_country]['UTILITY'] += orders[i]['PRICE_USD']*orders[i]['DELIVERED_QUANTITY']*margins[orders[i]['GOAL_CATEGORY_NAME']]
        except:
            consolidatedData[client_id,source_country]['UTILITY'] += orders[i]['PRICE_USD']*orders[i]['DELIVERED_QUANTITY']*0.04 # No margin found

        consolidatedData[client_id,source_country]['ORDER_IDS'].add(orders[i]['ID'])
    else:
        try:
            consolidatedData[client_id,source_country] = {
                'GMV': orders[i]['PRICE_USD']*orders[i]['DELIVERED_QUANTITY'],
                'UTILITY': orders[i]['PRICE_USD']*orders[i]['DELIVERED_QUANTITY']*margins[orders[i]['GOAL_CATEGORY_NAME']],
                'ORDER_IDS': {orders[i]['ID']},
                'FIRST_TIME': orders[i]['CREATED_AT']
            }
        except:
            consolidatedData[client_id,source_country] = {
                'GMV': orders[i]['PRICE_USD']*orders[i]['DELIVERED_QUANTITY'],
                'UTILITY': orders[i]['PRICE_USD']*orders[i]['DELIVERED_QUANTITY']*0.04,
                'ORDER_IDS': { orders[i]['ID'] },
                'FIRST_TIME': orders[i]['CREATED_AT']
            }

dropIdx=[]
for i in consolidatedData:
    if len(consolidatedData[i]['ORDER_IDS']) >= 5:
        timeDiference = (datetime.datetime.now()-consolidatedData[i]['FIRST_TIME']).total_seconds() / 60/ 60/24/365 # In Years
        constant = 0.7/(1+0.12-0.7)
        consolidatedData[i]['LTV'] = consolidatedData[i]['UTILITY']/timeDiference*constant
    else:
        dropIdx.append(i)
print(len(consolidatedData))
for i in dropIdx:
    consolidatedData.pop(i)
print(len(consolidatedData))

9036
5889


In [57]:
df = pd.DataFrame.from_dict(consolidatedData, orient='index')
df.to_excel('outputLTV.xlsx')

ctx = snowflake.connector.connect(
  user=SF_USERNAME,    #username,
  password=SF_PASSWORD,    #password,
  account=SF_ACCOUNT,
  warehouse=SF_WH,
  database='ANALYTICS',
  schema='PROD_STAGING'
)

cur=ctx.cursor()
sql = "CREATE OR REPLACE TABLE stg_py_clients_ltv ( \
                source_country varchar(1020), \
                client_id number(38,0), \
                ltv  float ) "

cur.execute(sql)
#DataFrame of the needed results to update in the same orde
# multiple records to be updated in tuple format
sql = "INSERT INTO stg_py_clients_ltv VALUES"
count = 0
for i in consolidatedData:
    count += 1
    sql += "(" \
            + "'" + i[1] + "'" + "," \
            + "" + str(i[0]) + "" + "," \
            + "" + str(consolidatedData[i]['LTV']) + "" \
            + ")"
    if (count % 10000 == 0):
                cur.execute(sql)
                print("Flushing 10000 records: ", count)
                sql = "INSERT INTO stg_py_clients_ltv VALUES "
    else:
        if count < len(consolidatedData):
            sql += ","

cur.execute(sql)    
print("Records updated successfully", count)
cur.close()

ProgrammingError: 000904 (42000): SQL compilation error: error line 1 at position 125
invalid identifier 'NAN'